## Libraries 

In [0]:

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Mounting google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd drive/My Drive/Colab Notebooks/cropped_months/

/content/drive/My Drive/Colab Notebooks/cropped_months


## Data Preparation & Transformations

In [0]:
train_dir = 'train'

test_dir = 'test'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
  
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
  
  
trainloader = torch.utils.data.DataLoader(train_data,batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True)
  
  
  
print(trainloader.dataset.classes)

['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']


## Getting Pre-trained ResNet Model

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

Putting gradient False so that the propogation won't change the pre-trained model

In [0]:
for param in model.parameters():
    param.require_grad = False

## Changing the FC layer of ResNet 

In [0]:
model.fc = nn.Linear(2048, 12)
# Freeze parameters so we don't backprop through them


## Loss Function & Optimizer

In [0]:
Epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [0]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

## Training

In [0]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = model(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()              #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.item()))
        
    torch.save(model.state_dict(), 'ResNET_ft.pth')

print('Finished Training')


0it [00:00, ?it/s]
Train Epoch: 0 [0/6000 (0%)]	Loss: 2.499332: : 0it [00:00, ?it/s]
Train Epoch: 0 [0/6000 (0%)]	Loss: 2.499332: : 1it [00:00,  1.88it/s]
Train Epoch: 0 [8/6000 (0%)]	Loss: 2.495344: : 1it [00:00,  1.88it/s]
Train Epoch: 0 [8/6000 (0%)]	Loss: 2.495344: : 2it [00:00,  2.42it/s]
Train Epoch: 0 [16/6000 (0%)]	Loss: 2.517237: : 2it [00:02,  2.42it/s]
Train Epoch: 0 [16/6000 (0%)]	Loss: 2.517237: : 3it [00:02,  1.33it/s]
Train Epoch: 0 [24/6000 (0%)]	Loss: 2.500724: : 3it [00:02,  1.33it/s]
Train Epoch: 0 [24/6000 (0%)]	Loss: 2.500724: : 4it [00:02,  1.45it/s]
Train Epoch: 0 [32/6000 (1%)]	Loss: 2.575611: : 4it [00:03,  1.45it/s]
Train Epoch: 0 [32/6000 (1%)]	Loss: 2.575611: : 5it [00:03,  1.35it/s]
Train Epoch: 0 [40/6000 (1%)]	Loss: 2.468442: : 5it [00:04,  1.35it/s]
Train Epoch: 0 [40/6000 (1%)]	Loss: 2.468442: : 6it [00:04,  1.26it/s]
Train Epoch: 0 [48/6000 (1%)]	Loss: 2.477256: : 6it [00:05,  1.26it/s]
Train Epoch: 0 [48/6000 (1%)]	Loss: 2.477256: : 7it [00:05,  1.07

Finished Training


## Testing Phase



In [0]:

print(trainloader.dataset.classes)

['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']


In [0]:
dataiter = iter(testloader)
images, labels = dataiter.next()

images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = model(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % trainloader.dataset.classes[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % trainloader.dataset.classes[labels[j]]
                              for j in range(len(images))))

Predicted:    sep   oct   dec   apr   jun   apr   jul   oct
Ground Truth:    sep   oct   dec   apr   jun   apr   jul   oct


## Calculating Accuracy : 98 %

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1440 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 1440 test images: 98 %
